In [1]:
import pandas as pd
df = pd.read_excel("conversation.xlsx")
df = df[df['대분류'] == '비즈니스']
df = df[df['소분류']=='회의']
df = df[df['상황'] == '의견 교환하기']

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

from tensorflow.keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kou81\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
# 이것은 일종의 매퍼역할이다. 인덱스를 일치시켜준다는 개념으로 사용하고 import 할 데이터를 이거와 매칭시켜주는 것으로 만들자.
real_text = []
lines_en = []
lines_ko = []
for i in range(101):
    real_text.append(df['번역문'][i])
    lines_en.append(df['번역문'][i])

for j in range(101):
    real_text.append(df['원문'][j])
    lines_ko.append(df['원문'][j])

tokenizer = Tokenizer()

tokenizer.fit_on_texts(real_text)
tkIndex = tokenizer.word_index
print(tkIndex)

{'the': 1, 'to': 2, 'we': 3, 'i': 4, 'it': 5, 'is': 6, 'of': 7, 'a': 8, 'you': 9, 'for': 10, 'be': 11, 'then': 12, 'that': 13, '네': 14, 'on': 15, 'how': 16, 'about': 17, 'if': 18, 'by': 19, 'in': 20, 'them': 21, 'new': 22, 'was': 23, 'there': 24, '그럼': 25, 'sure': 26, 'need': 27, 'but': 28, 'can': 29, 'as': 30, 'room': 31, 'email': 32, 'should': 33, '더': 34, '것': 35, "i'll": 36, 'this': 37, 'are': 38, 'our': 39, 'do': 40, 'yes': 41, "it's": 42, 'now': 43, 'will': 44, '이번': 45, '오늘': 46, 'and': 47, 'meeting': 48, 'more': 49, 'with': 50, 'which': 51, 'think': 52, 'use': 53, 'send': 54, 'get': 55, 'check': 56, 'one': 57, 'warehouse': 58, 'what': 59, '다시': 60, '주': 61, '가장': 62, '있나요': 63, '것이': 64, '같아요': 65, '바로': 66, 'have': 67, 'double': 68, 'at': 69, 'last': 70, 'would': 71, 'or': 72, 'probably': 73, 'company': 74, 'tomorrow': 75, '5': 76, 'so': 77, 'know': 78, 'me': 79, 'people': 80, '있습니다': 81, '그렇다면': 82, '새로운': 83, '할': 84, '많이': 85, '저는': 86, '내일': 87, '지금': 88, '있으면': 89, '메일로':

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = tokenizer.texts_to_sequences(real_text)
padded = pad_sequences(sequences,padding = "post")

# 확인
print(real_text[0])
print(sequences[0])
print(padded[0])

print(len(padded[0]))

How is the market's reaction to the newly released product?
[16, 6, 1, 289, 290, 2, 1, 291, 151, 152]
[ 16   6   1 289 290   2   1 291 151 152   0   0   0   0   0   0   0   0
   0   0   0]
21


In [5]:
lines_en = tokenizer.texts_to_sequences(lines_en)
lines_ko = tokenizer.texts_to_sequences(lines_ko)

In [6]:
# en & ko 글자 집합 구축
en_vocab = set()
for line in lines_en :
  for char in line:
    en_vocab.add(char)

print(en_vocab)

ko_vocab = set()
for line in lines_ko :
  for char in line:
    ko_vocab.add(char)

print(ko_vocab)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 

In [7]:
# en_vocab_size = len(en_vocab)+1
# ko_vocab_size = len(ko_vocab)+1
en_vocab_size = 513
ko_vocab_size = 935
print(en_vocab_size)
print(ko_vocab_size)

513
935


In [8]:
print(lines_ko[0])

[45, 127, 514, 232, 515, 233, 234]


In [9]:
# max length
max_en_len = max([len(line) for line in lines_en])
max_ko_len = max([len(line) for line in lines_ko])

print(max_en_len)
print(max_ko_len)

21
11


In [10]:
encoder_input = lines_en
decoder_input = lines_ko
decoder_target = lines_ko

In [11]:
# 최대 길이를 10으로 나누어 사용 (학습 시간 단축)\n",
max_en_len //= 10
max_ko_len //= 10
print(max_en_len) 
print(max_ko_len)  
encoder_input = pad_sequences(encoder_input, maxlen=max_en_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_ko_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_ko_len, padding='post')

2
1


In [12]:
# 원핫 벡터
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# **seq2seq 모델** **구현**

In [13]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [14]:
encoder_inputs = Input(shape=(None, en_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c] #은닉 상태, 셀 상태

In [15]:
decoder_inputs = Input(shape=(None, ko_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(ko_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

#adams
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

# embedding 계층 관련 학습!

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 513)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 935)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 788480      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  1220608     input_2[0][0]                    
                                                                 lstm[0][1]                   

In [16]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50


ValueError: in user code:

    C:\Users\kou81\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\kou81\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kou81\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kou81\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kou81\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\kou81\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\k